# This code estimates irrigation requirement for Soybean  using Penman-Monteith method

# 1. FAO Penman-Montieth method for estimating Ref. Evapotranspiration

In [ ]:
import pandas as pd
import datetime
import numpy as np
import math
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_excel('../Irrigation_Scheduling_Field70_2023.xlsx', 'Penman', skiprows=2)
df=df.iloc[:, : 6] 
df['Date'] = pd.to_datetime(df['Date'])
df["month1"]=df["Date"].dt.month
df

In [ ]:
# monthly_averages = df.groupby(["month1"]).aggregate({"Air Temperature":np.mean})
#df2 = monthly_averages.rename(columns={'Air Temperature':"Temp_avg"})
#df2

In [ ]:
data = {'month1': np.arange(1,13),
        'Temp_avg': [-3.150,-1.345,4.547,11.109,17.016,21.745,22.969,21.738,18.330,11.936,5.572,-0.620]}
 
# Create DataFrame
df2 = pd.DataFrame(data)

# Create a new column and calculate its values using if-else conditions
df2['heat_Flux'] = 0.07 * np.where(df2['month1'] == 1, 
                                    df2['Temp_avg'].shift(-1) - df2['Temp_avg'].iloc[-1],
                                    np.where(df2['month1'] == 12,
                                             df2['Temp_avg'].iloc[0] - df2['Temp_avg'].shift(1),
                                             df2['Temp_avg'].shift(-1) - df2['Temp_avg'].shift(1)))
df=pd.merge(df, df2, left_on='month1', right_on='month1')
df

In [ ]:
df["year"]=df["Date"].dt.year
df['month'] = 1
df['day'] = 1
df['J_Date']=pd.to_datetime(df[["year", "month", "day"]])
df['j']=df['Date'].apply(pd.Timestamp.to_julian_date)-df['J_Date'].apply(pd.Timestamp.to_julian_date)+1


In [ ]:
df = df.rename(columns={'Solar Radiation':'radiation','Air Temperature':"Temp",'Wind Speed':'Wind_speed','Vapor Pressure':'V_Pressure'})

#df=df.iloc[0:10,:]


In [ ]:
def PET_Penman (Wind_speed, radiation, Temp, V_Pressure, heat_Flux, j):
    lammbda= 2.501-Temp*0.002361 
     
    if Temp > 0:
        Sat_VP = 0.6108 * math.exp(17.27 * Temp / (237.3 + Temp))
    else:
        Sat_VP = V_Pressure * 0.1
        
    vpd= Sat_VP-V_Pressure*0.1
    slope= 4098*0.6108*math.exp(((17.27*Temp)/(Temp+237.3)))/((Temp+237.3)**2)
    
    Pressure= 101.3*((293-0.0065*213.36)/293)**5.26
    gamma  =0.001013*Pressure/(0.622*lammbda)
    sigma=4.903*10**-9
    
    Rbo= sigma*(Temp+273)**4
    Rs= radiation*60*60*24/10e5    
    dr=1+0.033*math.cos(2*math.pi*j/365)
    declination=0.409*math.sin((2*math.pi*j/365)-1.39)
    sunset_angle=math.acos(-1*math.tan(40.469794*math.pi/180)*math.tan(declination))
    Ra=(24*60*0.082/math.pi)*dr*(math.sin(40.469794*math.pi/180)*math.sin(declination)+math.cos(40.469794*math.pi/180)*math.cos(declination)*math.sin(sunset_angle))
    Rso=(213.36*0.00002+0.75)*Ra    
    Rnl=Rbo*(0.34-0.14*math.sqrt(0.1*V_Pressure))*(1.35*(Rs/Rso) - 0.35)      
    Rnet=Rs*(1-0.23)-Rnl
    ETo=(0.408*slope*(Rnet-heat_Flux)+gamma*(900/(Temp+273))*Wind_speed*vpd)/(slope+gamma*(1+0.34*Wind_speed))  
    ETo_in=ETo/25.4
    #return (lammbda,Sat_VP,vpd,slope,Pressure,gamma,sigma,Rbo,Rs,j,dr,declination,sunset_angle,Ra, Rso,Rnl,ETo)
    return ETo, ETo_in


In [ ]:
#df['ETo'] = df.apply(lambda row: PET(row['Wind_speed'], row['heat_Flux'], row['radiation'], row['Temp'], row['V_Pressure'],row['j']), axis=1)

# zip allows for multiple columns
%timeit df['ETo_mmDay'], df['ETo_in'] = zip(*df.apply(lambda row: PET_Penman(row['Wind_speed'], row['radiation'], row['Temp'], row['V_Pressure'], row['heat_Flux'],row['j']), axis=1))

#df.ETo_in=df.ETo_in.round(3)
df

## The Ref ET estimated using Penman-Monteith:

In [ ]:
#df.loc[(df.Date >= '2023-08-30') & (df.Date <= '2023-09-10')]
#df=df[['Date','Wind_speed','Precipitation','radiation','Temp','V_Pressure','ETo_mmDay','ETo_in']]
df=df[['Date','Precipitation','ETo_in']]
df['Precipitation']=df['Precipitation']/25.4
df

# 2. Crop Coefficent: Soybean 

In [ ]:
df_Kc = pd.read_excel('../Irrigation_Scheduling_Field70_2023.xlsx', 'Soy Irr', skiprows=6)
df_Kc=df_Kc.iloc[1:168, : 14] 
df_Kc.tail()

In [ ]:
df_Kc=df_Kc[['Date','Soil-Water Deficit Percent (Adjusted)\n(SWDPadj)']]

#df_Kc = df_Kc.rename(columns={'Week\nPast\nEmer-gence\n(WPE)':'WPE','Soil-Water Deficit Percent\n(SWDP)':'SWDP'})
df_Kc = df_Kc.rename(columns={'Soil-Water Deficit Percent (Adjusted)\n(SWDPadj)':'SWDPadj'})
df_Kc['SWDPadj'] = df_Kc['SWDPadj'].fillna(0)
df_Kc

In [ ]:
df_Kc['Date']=pd.to_datetime(df_Kc['Date']).dt.date
df_Kc['Date'] = pd.to_datetime(df_Kc['Date'])
df_Kc.head()

In [ ]:
Emergence = pd.to_datetime(["5/16/2023"]) # actual date is 5/13/2023 & date are in mm-dd-yy 23:59 format. This enable to get actual RZ value
Harvest = pd.to_datetime(["10/1/2023"])


In [ ]:
def calculate_WPE(date):
    if date < Emergence:
        return 0
    elif date > Harvest:
        return 0
    else:
        return ((((date-Emergence).days)/7).astype(int)+1)[0]

df_Kc['WPE']=df_Kc['Date'].apply(calculate_WPE)
df_Kc

## 2.1 Average Weekly crop coefficient for Soybean in Indiana

In [ ]:
Soybean_Kc_Indiana= {'Tmax_F': [1] * 22 ,
    'WPE': np.arange(1,23),
        'Kc_avg': [0.138, 0.181, 0.2367, 0.29667, 0.36333, 0.4533, 0.5589, 0.6511, 0.74, 0.8233, 0.9044, 0.9767, 0.9978, 0.9422, 0.87, 0.7867, 0.7211, 0.66, 0.1, 0.1, 0.1, 0.1]}

Soybean_Kc_Indiana = pd.DataFrame(Soybean_Kc_Indiana)
#Corn_Kc_Indiana["new"]= Corn_Kc_Indiana['Kc_avg'].shift(1)# delete it
Soybean_Kc_Indiana.tail()

In [ ]:
df_Kc=df_Kc.merge(Soybean_Kc_Indiana, on='WPE', how='left')

df_Kc = df_Kc.drop('Tmax_F', axis=1)
df_Kc['Kc_avg'] = df_Kc['Kc_avg'].fillna(0)
df_Kc

# 3. Merge Penman Monteith ETo and Soybean weekly Kc

In [ ]:
dat=df_Kc.merge(df, on='Date', how='left')
dat=dat.rename(columns={'Precipitation':'Eff_Rain'})
#dat=dat.iloc[0:130] # to reomve nan rows   
dat['Eff_Rain'] = dat['Eff_Rain'].fillna(0)

In [ ]:
if dat.index.max() > 129:   
    dat.loc[dat.index > 129, 'ETo_in'] = dat.iloc[129]['ETo_in']
dat   

In [ ]:
WA_rate = 0.045632799 # water applicationr ate (in/hr)- ned to calculate
RZinitial=4
RZMD=24  # Root Zone Maximum Depth (in)
W_RZMD=7 # week of RZMD
Emergence = pd.to_datetime(["5/15/2023"]) # actual date is 5/16/2023 & date are in mm-dd-yy 23:59 format. This enable to get actual RZ value

In [ ]:
# check spreadsheet how RZ max is 24. Therefore, i use a if condition for rz>24, rz=24
def RZ_all (Date, WPE):
    
    if Date <= Emergence:
        RZ=RZinitial
    elif Date > Harvest:
        RZ=RZinitial
    elif Date > Emergence and WPE < RZMD:
        RZ= RZ_previous_step + (RZMD - RZinitial) / ((W_RZMD- 1) * 7)
    else:
        RZ=RZMD
    
    if RZ>24:
        RZ=24
    else:
        RZ=RZ
        
    return RZ   

for index, row in dat.iterrows():
    RZ = RZ_all(row['Date'], row['WPE'])
    # if index==1:
    #     RZ_previous_step=4
    # else:
    #     RZ_previous_step = RZ    
    RZ_previous_step = RZ   
    
    dat.at[index, 'RZ'] = RZ
    
#df_Kc.iloc[51:70]
dat['eff_irr']=0
dat

In [ ]:
Soil_table = pd.DataFrame({
    'Layer': np.arange(0,8),
    'From': [0,0,6,12,18,24,48,48],
    'To': [0,6,12,18,24,48,48,48],
    'Thickness': [0,6,6,6,6,0,0,0],
    'AWHC': [0,0.23,0.23,0.21,0.2,0,0,0]})


Soil_table['AWHCj']=Soil_table['Thickness']*Soil_table['AWHC']
Soil_table['AWHCj_cum']=Soil_table['AWHCj'].cumsum()
Soil_table1=Soil_table[['To','AWHCj_cum']]

In [ ]:
def AWHCrz_func(rz):
    
    if rz in Soil_table1['To'].values:
        i = Soil_table1.index[Soil_table1['To'] == rz][0]
        dfx = Soil_table1.iloc[i:i+2]
    else:
        Soil_table1["To"] > rz
        v = Soil_table1["To"] > rz 
        i = v[v].index[0]   
        dfx = Soil_table1.iloc[i-1:i+1]
      
    
    AWHCrz= ((rz-dfx.iloc[0][0])/(dfx.iloc[1][0]-dfx.iloc[0][0]))* (dfx.iloc[1][1]-dfx.iloc[0][1])+dfx.iloc[0][1]        
            
    return AWHCrz

dat['AWHCrz'] = dat.apply(lambda x: AWHCrz_func(x['RZ']), axis=1)

dat

In [ ]:
SWDPcritical=0.7 #SWD (%) level beyond which ET is reduced due to drought stress
SWDP_1=0
SWD_1=0
WL=0

def Kc_Corn (Date, WPE, Kc_avg, ETo_in, AWHCrz, Eff_Rain, eff_irr, SWDPadj):
    
    # if WPE > 0:
    #     if SWDP_1 <= SWDPcritical:
    #         Kc=1
    #     else:
    #         Kc=Kc_avg #Kc=((1-SWDP_1)/(1-SWDPcritical))*Kc_avg --? this is the equation but used same table of kc avergae
    # else:
    #     Kc=0.2
     
    if WPE == 0:
        Kc=0.2
    else:
        Kc=Kc_avg 
    
    unstressed_ET=Kc*ETo_in   
    ET=Kc*ETo_in    
       
    
    
    if (SWD_1+ET-Eff_Rain-eff_irr) < 0:
        WL= -SWD_1-ET+Eff_Rain+eff_irr
    else:
        WL=0    
        
    if SWDPadj != 0:
        SWDvalue = SWDPadj * AWHCrz
    else:        
        SWDvalue = SWD_1+ET+WL-Eff_Rain-eff_irr 
    
    SWDP=SWDvalue/AWHCrz
    
    if SWDP_1 > 0.3:
        Irr_len= ET/WA_rate
    else:
        Irr_len=0 
        
        
    OUTPUT=(WL, ET,   Kc, Irr_len, SWDvalue, SWDP) 


    return OUTPUT


for index, i in dat.iterrows():
   
    OUT = Kc_Corn( i['Date'], i['WPE'], i['Kc_avg'], i['ETo_in'], i['AWHCrz'],i['Eff_Rain'], i['eff_irr'],i['SWDPadj'])  
    SWD_1 = OUT[4]  
    SWDP_1 = OUT[5]   
    
    OUT = pd.DataFrame(OUT)
    OUT=OUT.T
    OUT.columns = ["WL", "ET",'Kc','Irr_len','SWD','SWDP']
  
   #dat.at[index, 'SWD'] = SWD
    dat.at[index, 'WL' ] = OUT.iloc[0][0]   
    dat.at[index, 'ET' ] = OUT.iloc[0][1]
    dat.at[index, 'Kc' ] = OUT.iloc[0][2]
    dat.at[index, 'Irr_len' ] = OUT.iloc[0][3]
    dat.at[index, 'SWD'] = OUT.iloc[0][4]
    dat.at[index, 'SWDP'] = OUT.iloc[0][5]
    
dat.to_csv('../out_soy.csv', index=True)  


In [ ]:
dat['Total_ET'] = dat['ET'].cumsum()
dat['Total_Rain'] = dat['Eff_Rain'].cumsum()
dat['Total_Irr'] = dat['eff_irr'].cumsum()
dat['Total_WL'] = dat['WL'].cumsum()
dat.tail()            


## Need to check:
1. check effective irrigation  & hence affected WL, total irrigation, total water loss
2. KC value takes from weekly average kc value